In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#from source.utils import *
from source.classes import *
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm

# Set up and design the project

In [ ]:
with open('../../Desktop/DLC_social_1/DLC_social_1_exp_conditions.pickle', 'rb') as handle:
    Treatment_dict = pickle.load(handle)

In [ ]:
#Which angles to compute?
bp_dict = {'B_Nose':['B_Left_ear','B_Right_ear'],
          'B_Left_ear':['B_Nose','B_Right_ear','B_Center','B_Left_flank'],
          'B_Right_ear':['B_Nose','B_Left_ear','B_Center','B_Right_flank'],
          'B_Center':['B_Left_ear','B_Right_ear','B_Left_flank','B_Right_flank','B_Tail_base'],
          'B_Left_flank':['B_Left_ear','B_Center','B_Tail_base'],
          'B_Right_flank':['B_Right_ear','B_Center','B_Tail_base'],
          'B_Tail_base':['B_Center','B_Left_flank','B_Right_flank']}

In [ ]:
%%time
DLC_social_1 = project(path='../../Desktop/DLC_social_1/',#Path where to find the required files
                   smooth_alpha=0.85,                    #Alpha value for exponentially weighted smoothing
                   distances=['B_Center','B_Nose','B_Left_ear','B_Right_ear','B_Left_flank',
                              'B_Right_flank','B_Tail_base'],
                   ego=False,
                   angles=True,
                   connectivity=bp_dict,
                   arena='circular',                  #Type of arena used in the experiments
                   arena_dims=[380],                  #Dimensions of the arena. Just one if it's circular
                   video_format='.mp4',
                   table_format='.h5',
                   exp_conditions=Treatment_dict)

# Run project

In [ ]:
%%time
DLC_social_1_coords = DLC_social_1.run(verbose=True)
print(DLC_social_1_coords)
type(DLC_social_1_coords)

# Generate coords

In [ ]:
%%time
ptest = DLC_social_1_coords.get_coords(center=True, polar=False, speed=0, length='00:10:00')
ptest._type

In [ ]:
%%time
dtest = DLC_social_1_coords.get_distances(speed=0, length='00:10:00')
dtest._type

In [ ]:
%%time
atest = DLC_social_1_coords.get_angles(degrees=True, speed=0, length='00:10:00')
atest._type

# Visualization playground

In [ ]:
ptest.plot_heatmaps(['B_Center', 'W_Center'], i=1)

In [ ]:
#Plot animation of trajectory over time with different smoothings
plt.plot(ptest['Day2Test13DLC']['B_Center'].iloc[:5000]['x'],
         ptest['Day2Test13DLC']['B_Center'].iloc[:5000]['y'], label='alpha=0.85')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Mouse Center Trajectory using different exponential smoothings')
plt.legend()
plt.show()

# Dimensionality reduction playground

In [ ]:
pca = ptest.pca(4, 1000)

In [ ]:
plt.scatter(*pca[0].T)
plt.show()

# Preprocessing playground

In [ ]:
mtest = merge_tables(DLC_social_1_coords.get_coords(center=True, polar=True, length='00:10:00'))#,
#                      DLC_social_1_coords.get_distances(speed=0, length='00:10:00'),
#                      DLC_social_1_coords.get_angles(degrees=True, speed=0, length='00:10:00'))

In [ ]:
#pptest = mtest.preprocess(window_size=51, filter='gaussian', sigma=10, shift=20)

In [ ]:
pttest = mtest.preprocess(window_size=51, filter=None)
pttest.shape

In [ ]:
plt.plot(pttest[2,:,2], label='normal')
plt.plot(pptest[2,:,2], label='gaussian')
plt.legend()
plt.show()

# Trained models playground

### Seq 2 seq Variational Auto Encoder

In [ ]:
pttest = pttest[:1000]

In [ ]:
CONV_filters = 64
LSTM_units_1 = 128
LSTM_units_2 = 64
DENSE_1 = 64
DENSE_2 = 32
ENCODING = 20
DROPOUT_RATE = 0.2

original_dim = pttest.shape[1:]
batch_size = 256

In [ ]:
from source.hypermodels import *
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Lambda, Bidirectional, LSTM
from tensorflow.keras import backend as K

# Encoder Layers
Model_E0 = tf.keras.layers.Conv1D(
    filters=CONV_filters,
    kernel_size=5,
    strides=1,
    padding="causal",
    activation="relu",
)
Model_E1 = Bidirectional(
    LSTM(
        LSTM_units_1,
        activation="tanh",
        return_sequences=True,
        kernel_constraint=UnitNorm(axis=0),
    )
)
Model_E2 = Bidirectional(
    LSTM(
        LSTM_units_2,
        activation="tanh",
        return_sequences=False,
        kernel_constraint=UnitNorm(axis=0),
    )
)
Model_E3 = Dense(DENSE_1, activation="relu", kernel_constraint=UnitNorm(axis=0))
Model_E4 = Dense(DENSE_2, activation="relu", kernel_constraint=UnitNorm(axis=0))
Model_E5 = Dense(
            ENCODING,
            activation="relu",
            kernel_constraint=UnitNorm(axis=1),
            activity_regularizer=UncorrelatedFeaturesConstraint(3, weightage=1.0),
        )

# Decoder layers
Model_D4 = Bidirectional(
    LSTM(
        LSTM_units_1,
        activation="tanh",
        return_sequences=True,
        kernel_constraint=UnitNorm(axis=1),
    )
)
Model_D5 = Bidirectional(
    LSTM(
        LSTM_units_1,
        activation="sigmoid",
        return_sequences=True,
        kernel_constraint=UnitNorm(axis=1),
    )
)

# Define and instanciate encoder
x = Input(shape=original_dim)
encoder = Model_E0(x)
encoder = Model_E1(encoder)
encoder = Model_E2(encoder)
encoder = Model_E3(encoder)
encoder = Dropout(DROPOUT_RATE)(encoder)
encoder = Model_E4(encoder)
encoder = Model_E5(encoder)
z_mean = Dense(ENCODING)(encoder)
z_log_sigma = Dense(ENCODING)(encoder)

In [ ]:
def sampling(args, epsilon_std=1.):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=K.shape(z_mean),
                              mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_sigma) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
# so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
z = Lambda(sampling)([z_mean, z_log_sigma])

In [ ]:
# Define and instanciate decoder
decoder = DenseTranspose(Model_E5, activation="relu", output_dim=ENCODING)(z)
decoder = DenseTranspose(Model_E4, activation="relu", output_dim=DENSE_2)(decoder)
decoder = DenseTranspose(Model_E3, activation="relu", output_dim=DENSE_1)(decoder)
decoder = RepeatVector(pttest.shape[1])(decoder)
decoder = Model_D4(decoder)
decoder = Model_D5(decoder)
x_decoded_mean = TimeDistributed(Dense(original_dim[1]))(decoder)

In [ ]:
# end-to-end autoencoder
klvae = Model(x, x_decoded_mean)

# encoder, from inputs to latent space
encoder = Model(x, z_mean)

# generator, from latent space to reconstructed inputs
decoder_input = Input(shape=(ENCODING,))
decoder = DenseTranspose(Model_E5, activation="relu", output_dim=ENCODING)(decoder_input)
decoder = DenseTranspose(Model_E4, activation="relu", output_dim=DENSE_2)(decoder)
decoder = DenseTranspose(Model_E3, activation="relu", output_dim=DENSE_1)(decoder)
decoder = RepeatVector(pttest.shape[1])(decoder)
decoder = Model_D4(decoder)
decoder = Model_D5(decoder)
x_decoded_mean = TimeDistributed(Dense(original_dim[1]))(decoder)
generator = Model(decoder_input, x_decoded_mean)

In [ ]:
vae.summary()

In [ ]:
tf.keras.utils.plot_model(vae)

In [ ]:
def compute_kernel(x, y):
    x_size = K.shape(x)[0]
    y_size = K.shape(y)[0]
    dim = K.shape(x)[1] * K.shape(x)[2]
    tiled_x = K.tile(K.reshape(x, K.stack([x_size, 1, dim])), K.stack([1, y_size, 1]))
    tiled_y = K.tile(K.reshape(y, K.stack([1, y_size, dim])), K.stack([x_size, 1, 1]))
    return K.exp(-tf.reduce_mean(K.square(tiled_x - tiled_y), axis=2) / K.cast(dim, tf.float32))

def compute_mmd(x, y, sigma_sqr=1.0):
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    xy_kernel = compute_kernel(x, y)
    return tf.reduce_mean(x_kernel) + tf.reduce_mean(y_kernel) - 2 * tf.reduce_mean(xy_kernel)

In [ ]:
from tensorflow.keras.losses import Huber

def vae_loss(x, x_decoded_mean):
    huber_loss = Huber(reduction="sum", delta=100.0)
    huber_loss = original_dim * huber_loss(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return K.mean(huber_loss + kl_loss[:, None])

def vae_mmd_loss(x, x_decoded_mean):
    huber_loss = Huber(reduction="sum", delta=100.0)
    huber_loss = original_dim * huber_loss(x, x_decoded_mean)
    mmd_loss = compute_mmd(x, x_decoded_mean)
    return huber_loss + mmd_loss

klvae.compile(optimizer='adam', loss=vae_loss, experimental_run_tf_function=False, metrics=['mae'])

In [ ]:
#tf.config.experimental_run_functions_eagerly(False)
ptrain = pttest[np.random.choice(pttest.shape[0], 1000, replace=False), :, :]
klhistory = klvae.fit(ptrain, ptrain, epochs=50, batch_size=batch_size, verbose=1)

In [ ]:
#plt.plot(history.history['mae'], label='Huber + MMD mae')
plt.plot(klhistory.history['mae'], label='Huber + KL mae')
plt.plot(hhistory.history['mae'], label='Huber mae')

plt.legend()
plt.show()

In [ ]:
#Huber loss + MMD/ELBO in training data
plt.plot(pttest[:500,0,0], label='data')
#plt.plot(vae.predict(pttest[:500])[:,0,0], label='MMD reconstruction')
plt.plot(klvae.predict(pttest[:500])[:,0,0], label='ELBO reconstruction')
plt.plot(hvae.predict(pttest[:500])[:,0,0], label='Huber reconstruction')

plt.legend()
plt.show()